In [ ]:
!pip install tensorflow scikeras scikit-learn
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, ParameterSampler, StratifiedKFold
from sklearn.metrics import (
    precision_score, precision_recall_curve, average_precision_score,
    classification_report, roc_auc_score, confusion_matrix, matthews_corrcoef
)
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow.keras.backend as K
import joblib
import random

# --- Load & split ---
df = pd.read_csv("bcsc_concatenated_no_9.csv")
X = df.drop(columns="breast_cancer_history")
y = df["breast_cancer_history"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.20,
    stratify=y,
    random_state=42
)

# --- Class weights matching your XGB setup ---
scale_pos_weight = len(y_train[y_train == 0]) / len(y_train[y_train == 1])
pos_penalty_factor = 10.0
w_neg = 1.0
w_pos = scale_pos_weight * pos_penalty_factor

def weighted_bce(y_true, y_pred):
    w = y_true * w_pos + (1 - y_true) * w_neg
    bce = K.binary_crossentropy(y_true, y_pred)
    return K.mean(w * bce)

# --- Model factory ---
def create_model(hidden_units, dropout_rate, learning_rate):
    m = Sequential([
        Dense(hidden_units, activation="relu", input_dim=X_train.shape[1]),
        Dropout(dropout_rate),
        Dense(hidden_units // 2, activation="relu"),
        Dropout(dropout_rate),
        Dense(1, activation="sigmoid"),
    ])
    m.compile(
        loss=weighted_bce,
        optimizer=Adam(learning_rate=learning_rate),
        metrics=["AUC"]
    )
    return m

# --- Hyperparameter space & sampler ---
param_dist = {
    "hidden_units":   [16, 32, 64],
    "dropout_rate":   [0.1, 0.2, 0.3],
    "learning_rate":  [1e-3, 5e-4, 1e-4],
    "epochs":         [30, 50],
    "batch_size":     [16, 32],
}

n_iter = 10
random.seed(42)
sampler = list(ParameterSampler(param_dist, n_iter=n_iter, random_state=42))

# --- 3-fold stratified CV to pick best params by precision at default 0.5 ---
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

best_score = -np.inf
best_params = None

for idx, params in enumerate(sampler, 1):
    cv_scores = []
    for train_idx, val_idx in skf.split(X_train, y_train):
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
        
        model = create_model(
            hidden_units=params["hidden_units"],
            dropout_rate=params["dropout_rate"],
            learning_rate=params["learning_rate"]
        )
        # no early stopping in CV
        model.fit(
            X_tr, y_tr,
            epochs=params["epochs"],
            batch_size=params["batch_size"],
            verbose=0
        )
        y_pred = model.predict(X_val).flatten()
        y_cls = (y_pred >= 0.5).astype(int)
        cv_scores.append(precision_score(y_val, y_cls))
    
    mean_prec = np.mean(cv_scores)
    print(f"Trial {idx}/{n_iter}: prec={mean_prec:.4f} ← {params}")
    if mean_prec > best_score:
        best_score, best_params = mean_prec, params

print("\nBest params (by CV precision):")
print(best_params, "→ precision", best_score)

# --- Retrain on full train set with EarlyStopping ---
model = create_model(
    hidden_units=best_params["hidden_units"],
    dropout_rate=best_params["dropout_rate"],
    learning_rate=best_params["learning_rate"]
)
es = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
model.fit(
    X_train, y_train,
    epochs=best_params["epochs"],
    batch_size=best_params["batch_size"],
    validation_split=0.2,
    callbacks=[es],
    verbose=2
)

# --- Get probabilities on test set ---
y_prob = model.predict(X_test).flatten()

# --- Precision‐Recall analysis & threshold selection ---
precision, recall, thresholds = precision_recall_curve(y_test, y_prob)
avg_prec = average_precision_score(y_test, y_prob)

target_prec = 0.7
valid = np.where(precision >= target_prec)[0]
if valid.size:
    i = valid[np.argmax(recall[valid])]
    thr = thresholds[i if i < thresholds.size else -1]
    print(f"\nAt precision ≥ {target_prec:.2f}, threshold={thr:.3f}, "
          f"prec={precision[i]:.3f}, recall={recall[i]:.3f}")
else:
    thr = 0.5
    print(f"\nNo threshold achieves precision ≥ {target_prec:.2f}, defaulting to 0.5")

# --- Plot PR curve ---
plt.figure(figsize=(8,6))
plt.plot(recall, precision, marker='.', label=f'NN (AP={avg_prec:.2f})')
if valid.size:
    plt.scatter(recall[i], precision[i], color='red',
                label=f'P={precision[i]:.2f}, R={recall[i]:.2f}, Thr={thr:.2f}')
plt.xlabel('Recall'); plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.grid(True); plt.legend(); plt.show()

# --- Final eval at chosen threshold ---
y_pred = (y_prob >= thr).astype(int)
print(classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred, labels=[0,1]))
print("ROC AUC:", roc_auc_score(y_test, y_prob))
print("Matthews Corr Coef:", matthews_corrcoef(y_test, y_pred))

# --- Save artifacts ---
os.makedirs("models", exist_ok=True)
model.save("models/bcsc_nn_model.h5")
joblib.dump(thr, "models/threshold.pkl")
print("\nModel and threshold saved.")




C:\Users\vetdd\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
!pip install xgboost


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df = pd.read_csv("METABRIC_removedgene.csv")
df = df.dropna()

df['status_combo'] = (
    df['er_status'].astype(str) + '_' +
    df['pr_status'].astype(str) + '_' +
    df['her2_status'].astype(str)
)

df['survival_group'] = np.where(
    df['overall_survival_months'] <= 72, 
    1, 
    2
)

avg_survival = (
    df.groupby('status_combo')['overall_survival_months']
      .mean()
      .reset_index(name='avg_survival_months')
)
avg_survival = avg_survival.sort_values('avg_survival_months')

plt.figure(figsize=(8, 5))
plt.bar(
    avg_survival['status_combo'],
    avg_survival['avg_survival_months']
)
plt.xlabel('ER/PR/HER2 Status Combination')
plt.ylabel('Average Overall Survival (months)')
plt.xticks(rotation=45, ha='right')
plt.title('Average Overall Survival by ER/PR/HER2 Combination')
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load & clean
df = pd.read_csv("METABRIC_removedgene.csv")
df = df.dropna(subset=['er_status', 'pr_status', 'her2_status', 'age_at_diagnosis'])

# Build combo and survival group (if still needed)
df['status_combo'] = (
    df['er_status'].astype(str) + '_' +
    df['pr_status'].astype(str) + '_' +
    df['her2_status'].astype(str)
)
df['survival_group'] = np.where(
    df['overall_survival_months'] <= 72, 1, 2
)

# Compute average age by combo
avg_age = (
    df.groupby('status_combo')['age_at_diagnosis']
      .mean()
      .reset_index(name='avg_age_years')
      .sort_values('avg_age_years')
)

# Plot bar chart in sorted order
plt.figure(figsize=(8, 5))
plt.bar(
    avg_age['status_combo'],
    avg_age['avg_age_years']
)
plt.xlabel('ER/PR/HER2 Status Combination')
plt.ylabel('Average Age at Diagnosis (years)')
plt.xticks(rotation=45, ha='right')
plt.title('Average Age at Diagnosis by ER/PR/HER2 Combination')
plt.tight_layout()
plt.show()


In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import (classification_report,roc_auc_score,confusion_matrix,matthews_corrcoef,make_scorer,precision_score,precision_recall_curve,average_precision_score
)
import xgboost as xgb
import joblib

# Load and split data 
df = pd.read_csv("bcsc_concatenated_no_9.csv")
X = df.drop(columns="breast_cancer_history")
y = df["breast_cancer_history"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.20,
    stratify=y,
    random_state=42
)

# Compute class weights
scale_pos_weight = len(y_train[y_train==0])/len(y_train[y_train==1])
pos_penalty_factor = 10.0
weight_neg = 1.0
weight_pos = scale_pos_weight*pos_penalty_factor
def weighted_logloss(y_true: np.ndarray, y_pred: np.ndarray):
    
    
    p = 1.0 / (1.0 + np.exp(-y_pred))
    
    w = np.where(y_true == 1, weight_pos, weight_neg)
    
    grad = w * (p - y_true)
    hess = w * p * (1.0 - p)
    return grad, hess

# Hyperparameter search setup
penalized_clf = xgb.XGBClassifier(
    objective=weighted_logloss,
    eval_metric="auc",
    random_state=42
)

param_dist = {
    "n_estimators":     [100, 300, 500],
    "max_depth":        [3, 5, 7],
    "learning_rate":    [0.01, 0.05, 0.1],
    "subsample":        [0.6, 0.8, 1.0],
    "colsample_bytree": [0.6, 0.8, 1.0],
    "gamma":            [0, 1, 5],
}

search = RandomizedSearchCV(
    estimator=penalized_clf,
    param_distributions=param_dist,
    n_iter=20,
    scoring=make_scorer(precision_score, pos_label=1),
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=-1
)
# Run hyperparameter search ---
search.fit(X_train, y_train)
print("Best hyperparameters:", search.best_params_)

# Retrain on full training set ---
best = search.best_estimator_
best.fit(X_train, y_train)

# Compute test‐set probabilities ---
y_prob = best.predict_proba(X_test)[:, 1]

# Precision‐Recall analysis ---
precision, recall, pr_thresholds = precision_recall_curve(y_test, y_prob)
avg_prec = average_precision_score(y_test, y_prob)

# pick threshold at precision ≥ 0.7
target_precision = 0.7
valid = np.where(precision >= target_precision)[0]
if len(valid) > 0:
    best_idx = valid[np.argmax(recall[valid])]
    matched_precision = precision[best_idx]
    matched_recall = recall[best_idx]
    matched_threshold = pr_thresholds[
        best_idx if best_idx < len(pr_thresholds) else -1
    ]
    print(f"At precision ≥ {target_precision:.2f}:")
    print(f"  Precision: {matched_precision:.3f}")
    print(f"  Recall:    {matched_recall:.3f}")
    print(f"  Threshold: {matched_threshold:.3f}")
else:
    print(f"No recall point found where precision ≥ {target_precision:.2f}")
    # plot Precision-Recall curve
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, marker='.', label=f'XGBoost (AP={avg_prec:.2f})')
plt.scatter(
    [matched_recall],
    [matched_precision],
    color='red',
    label=(
        f'Precision={matched_precision:.2f}\n'
        f'Recall={matched_recall:.2f}\n'
        f'Threshold={matched_threshold:.2f}'
    )
)
plt.xlabel('Recall', fontsize=14)
plt.ylabel('Precision', fontsize=14)
plt.title('Precision-Recall Curve', fontsize=16)
plt.grid(True)
plt.legend()
plt.show()

# --- Final evaluation at chosen threshold ---
best_thresh = matched_threshold
y_pred = (y_prob >= best_thresh).astype(int)

print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred, labels=[0,1])
print("Confusion Matrix:\n", cm)
print("\nTest ROC AUC:", roc_auc_score(y_test, y_prob))
print("Matthews Correlation Coefficient:", matthews_corrcoef(y_test, y_pred))
# --- Feature importances ---
feat_imp_df = pd.DataFrame({
    'feature': X_train.columns,
    'importance': best.feature_importances_
}).sort_values('importance', ascending=False)
print(feat_imp_df)

# plot top 15 feature importances
top_n = 15
fig, ax = plt.subplots(figsize=(8, 6))
feat_imp_df.head(top_n).plot.barh(x='feature', y='importance', legend=False, ax=ax)
plt.title('Top Feature Importances (XGBoost)')
plt.xlabel('Importance Score')
ax.invert_yaxis()
plt.tight_layout()
plt.savefig("feature_importance_xgb.jpg", dpi=300)
plt.close()

# --- Save model & threshold ---
os.makedirs("models", exist_ok=True)
joblib.dump(best, os.path.join("models", "bcsc_xgb_model.pkl"))
joblib.dump(best_thresh, os.path.join("models", "threshold.pkl"))
print("Model and threshold saved.")

